In [12]:
import argparse
import tensorflow as tf
import glovedata
from glovedata import FEATURES
#import tensorglove_osc_server

In [18]:
batch_size = 1000
train_steps = 1

In [19]:
def serving_input_receiver_fn():
    """Build the serving inputs."""
    inputs = {}
    for feat in FEATURES:
        inputs[feat] = tf.placeholder(shape=[None], dtype='float32')

    features = {
        key: tf.expand_dims(tensor, -1)
        for key, tensor in inputs.items()
    }

    return tf.estimator.export.ServingInputReceiver(features,
                                                    inputs)

In [20]:
# Fetch the data
(train_x, train_y), (test_x, test_y) = glovedata.load_data()

/Users/Mike/Documents/Code/side-projects/tensorglove/python/glovedata.py:53: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  data.convert_objects()
/Users/Mike/Documents/Code/side-projects/tensorglove/python/glovedata.py:58: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  train_y.convert_objects()
/Users/Mike/Documents/Code/side-projects/tensorglove/python/glovedata.py:60: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  test_y.convert_objects()


In [21]:
# Feature columns describe how to use the input.
my_feature_columns = []
for key in train_x.keys():
    my_feature_columns.append(tf.feature_column.numeric_column(key=key))

# Build 2 hidden layer DNN with 10, 10 units respectively.
classifier = tf.estimator.DNNClassifier(
    feature_columns=my_feature_columns,
    # Two hidden layers of 10 nodes each.
    hidden_units=[10, 10],
    # The model must choose between 4 classes.
    n_classes=4,
    model_dir="model")

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_steps': None, '_log_step_count_steps': 100, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x11a0274e0>, '_model_dir': 'model', '_is_chief': True, '_tf_random_seed': None, '_keep_checkpoint_max': 5, '_save_checkpoints_secs': 600, '_keep_checkpoint_every_n_hours': 10000, '_save_summary_steps': 100, '_num_ps_replicas': 0, '_global_id_in_cluster': 0, '_num_worker_replicas': 1, '_train_distribute': None, '_session_config': None, '_evaluation_master': '', '_master': '', '_service': None, '_task_type': 'worker', '_task_id': 0}


In [22]:
# Train the Model.
classifier.train(
    input_fn=lambda: glovedata.train_input_fn(train_x, train_y,
                                              batch_size),
    steps=train_steps)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from model/model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1001 into model/model.ckpt.
INFO:tensorflow:step = 1001, loss = 65.0034
INFO:tensorflow:Loss for final step: 65.0034.


In [24]:
# Evaluate the model.
eval_result = classifier.evaluate(
    input_fn=lambda: glovedata.eval_input_fn(test_x, test_y,
                                             batch_size))


print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-05-31-14:40:06
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from model/model.ckpt-1001
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-05-31-14:40:07
INFO:tensorflow:Saving dict for global step 1001: accuracy = 0.9751107, average_loss = 0.083439, global_step = 1001, loss = 78.06315

Test set accuracy: 0.975



In [25]:
# perform a sample prediction

# Generate predictions from the model
expected = ['Fist']
feature_values = [0, 0, 0, 1, 0.2298394, 0.9228836, -0.1887825, -0.2445833, 0.03349165, -0.2397906, -0.1800468,
                  0.9533949, -0.07355202, 0.2745002, -0.2481479, 0.9261007, -4.470348E-08, 0.2213377, 1.490116E-08,
                  0.9751973, -2.235174E-08, -5.960464E-08, 4.470348E-08, 1, 0.09148686, -0.09309785, -0.6949129,
                  0.7071485, 7.105427E-15, 7.45058E-08, -0.7191889, 0.6948146, 1.490116E-08, -1.490116E-08,
                  -0.566272, 0.8242185, -2.235174E-08, -5.960464E-08, 4.470348E-08, 1, -3.725291E-08, 5.960464E-08,
                  -0.7009093, 0.7132504, 1.490116E-08, 3.352762E-08, -0.772563, 0.6349381, -7.450572E-09,
                  2.980232E-08, -0.6148145, 0.7886717, -2.235174E-08, -5.960464E-08, 4.470348E-08, 1, -0.06108833,
                  0.06216392, -0.6982422, 0.7105362, -3.352762E-08, 1.490116E-08, -0.7539417, 0.6569413,
                  5.960465E-08, 5.960463E-08, -0.5976215, 0.8017784, -2.235174E-08, -5.960464E-08, 4.470348E-08, 1,
                  -0.1217117, 0.1238547, -0.6902609, 0.7024146, -5.029142E-08, 3.725291E-08, -0.819152, 0.5735765,
                  5.215406E-08, 5.960463E-08, -0.6427875, 0.7660446]
# the data is expected to be a list of feature values (as it is configured for batching)
predict_x = dict(zip(FEATURES, [[x] for x in feature_values]))

predictions = classifier.predict(
    input_fn=lambda: glovedata.eval_input_fn(predict_x,
                                             labels=None,
                                             batch_size=batch_size))

template = '\nPrediction is "{}" ({:.1f}%), expected "{}"'

for pred_dict, expec in zip(predictions, expected):
    class_id = pred_dict['class_ids'][0]
    probability = pred_dict['probabilities'][class_id]

    print(template.format(glovedata.GESTURES[class_id],
                          100 * probability, expec))

NameError: name 'abatch_size' is not defined

In [26]:
export_dir = classifier.export_savedmodel(
    export_dir_base="model_export",
    serving_input_receiver_fn=serving_input_receiver_fn)
print('Exported to:', export_dir)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Signatures INCLUDED in export for Predict: ['predict']
INFO:tensorflow:Signatures INCLUDED in export for Classify: None
INFO:tensorflow:Signatures EXCLUDED from export because they cannot be be served via TensorFlow Serving APIs:
INFO:tensorflow:'classification' : Classification input must be a single string Tensor; got {'Human_RightHandThumb3_Quat_X': <tf.Tensor 'Placeholder_16:0' shape=(?,) dtype=float32>, 'Human_RightInHandIndex3_Quat_X': <tf.Tensor 'Placeholder_32:0' shape=(?,) dtype=float32>, 'Human_RightInHandRing3_Quat_Z': <tf.Tensor 'Placeholder_66:0' shape=(?,) dtype=float32>, 'Human_RightHandThumb3_Quat_Z': <tf.Tensor 'Placeholder_18:0' shape=(?,) dtype=float32>, 'Human_RightHandThumb2_Quat_X': <tf.Tensor 'Placeholder_12:0' shape=(?,) dtype=float32>, 'Human_RightInHandIndex2_Quat_Z': <tf.Tensor 'Placeholder_30:0' shape=(?,) d